In [13]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
import itertools as it
import math
import sys
sys.path.append('../')
from agents.agent import Agent
from agents.utils.utility import sigmoid_update #, linear_update


### Idea
If I have a good estimate of your prior and of your error, can I learn your internal model M across time steps?
<br/>
Let's start by assuming I have a perfect estimate of your priors and error

In [4]:
# agent
a = Agent(state_size=1, model_var=0, prediction="sigmoid", behavior="sigmoid", attention="static", seed=7)
# error the agent is being fed
time = 20
#noise = np.random.uniform(0,1,50)
#print(noise)
#e_in = np.linspace(1, 0.1, time)
#e_in = e_in * noise
#print(e_in)
print(a.get_behav_model())


[0.97798951]


In [116]:
behaviors = np.random.choice([0, 1], size=(time,), p=[1./3, 2./3])
print("behaviors: ", behaviors)

behaviors:  [1 0 0 1 1]


In [117]:
# for each time step, send a behavior, get a prior, get error
errors = []
priors = []
m_hat = 0.5
for t in range(time):
    bp = a.get_behav_priors()
    priors.append(bp[0])
    a.get_world([behaviors[t]])
    dif, avg_abs_error = a.behavior_prediction_error()
    errors.append(dif[0])
    a.learn_conform()
    a.learn_predict_world()
print("errors: ", errors[:4])
print("priors: ", priors[:4])


errors:  [0.614421736848592, -0.5370530390093043, -0.40406191104371686, 0.6883958886461468]
priors:  [0.8442932825298948, 0.9081642599612204, 0.8539834046210683, 0.7975454113975015]


### Lets find the M to minimize the derivative of the loss function

In [118]:
def update_weights(m, X, Y, learning_rate):
    m_deriv = 0
    #b_deriv = 0
    N = len(X)
    for i in range(N-1):
        # Calculate partial derivatives
        # -2x(y - (mx + b))
        #m_deriv += -2*X[i] * (Y[i+1] - (m*X[i]+Y[i]))
        e = X[i]*m
        m_deriv -= ( (2*m*(1-Y[i])*np.exp(-e)) * (Y[i+1]-(1 / ((((1-Y[i])*np.exp(-e))/Y[i]) + 1))) / (Y[i]*( ((((1-Y[i])*np.exp(-e))/Y[i]) + 1) ** 2)) )
        #m_deriv += sigmoid_update(prev=Y[i], error=[e])

        # -2(y - (mx + b))
        #b_deriv += -2*(Y[i] - (m*X[i] + b))

    # We subtract because the derivatives point in direction of steepest ascent
    m -= (m_deriv / float(N-1)) * learning_rate
    #b -= (b_deriv / float(N)) * learning_rate

    return m

In [119]:
m_hat = update_weights(m_hat, errors, priors, .5)
m = a.get_behav_model()
print("m_hat: ", m_hat)
#print("b: ", b)
print("m: ", m)

m_hat:  0.5000719188695288
m:  [0.97798951]


### Linear Experiment
For the sake of simplicity, let's make some new assumptions:

1. priors are determined by a function at each time step that is always centered around the same set point/intercept
2. the prior function is linear, rather than a sigmoid
3. M is a coefficent of error and the prior is the intercept = new prior
4. for this experiment let's initially assume we have access to the prior of the other agent and are just trying to estimate their prior function

Why do this?

1. simpler estimation of latent variables M and B
2. can more eeffectivelly use conjugate priors to estimate B (beta distribution)

In [14]:
def linear_update(B, M, X):
    Xt = (M*X) + B
    if Xt > 1:
        return 1
    if Xt < 0:
        return 0
    return Xt

In [223]:
time = 1
state_size = 1
# for each time step, send a behavior, get a prior, get error
errors = np.linspace(0.9,0.1,time)
noise = np.random.uniform(0.7,0.9,time)
errors = errors * noise
print("errors:  ", errors)
base_priors = np.random.uniform(0.1,0.9,state_size)
print("base priors: ", base_priors)
M = np.random.uniform(-0.5,0.5,state_size)
print("M: ", M)


errors:   [0.69307937]
base priors:  [0.27810895]
M:  [-0.12080174]


In [224]:
priors = []
for t in range(time):
    p = linear_update(base_priors[t], M[0], errors[t])
    priors.append(p)
print("prior: ",priors)

prior:  [0.19438376357250664]


In [225]:
# I know the base prior from past time steps, the error because I assume it based on my prior, and I see your current prior
print("M = (new_prior - base_prior) / error")
print(f"{M[0]} = {priors[0]} - {base_priors[0]} / {errors[0]}")

M = (new_prior - base_prior) / error
-0.12080173532762395 = 0.19438376357250664 - 0.2781089542429256 / 0.693079370452333


### New Assumptions
Now let's assume we do not have access to the priors (base and updates).

Can we still estimate M?

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy

In [220]:
time = 10
state_size = 2
# for each time step, send a behavior, get a prior, get error
errors = np.linspace(0.6,0.1,time)
noise = np.random.uniform(-0.5,0.5,time)
errors = errors * noise
base_priors = np.random.uniform(0.1,0.9,state_size)
print("some pre-computed errors:  ", errors[:5])
print("base priors: ", base_priors)
M = np.random.uniform(-0.5,0.5,state_size)
print("Ms: ", M)


behaviors = []
for t in range(time):
    step = []
    for s in range(state_size):
        p = linear_update(base_priors[s], M[s], errors[s])
        behavior = np.random.binomial(1, p)
        step.append(behavior)
    behaviors.append(step)

some pre-computed errors:   [ 0.00192605  0.1040081  -0.21695504 -0.01443688  0.13642666]
base priors:  [0.7052405  0.43962337]
Ms:  [-0.4341728   0.43380865]


In [221]:
# get an estimate of the base priors
a = [2]*state_size
b = [2]*state_size
mode = [0.5]*state_size
m_hat = [0]*state_size
buckets = [{} for s in range(state_size)]
max_buckets = [0]*state_size
alpha = 0.1

for t in range(time):
    if t % 100 == 0:
        print(f"T: {t}, True Priors: {base_priors}, Posterior Estimate: {mode}")
        print(f"True M: {M}, M Estimate: {m_hat}")
        #print("\n")
    for s in range(state_size):
        a[s] = a[s]+behaviors[t][s]
        b[s] = b[s]-behaviors[t][s]+1
        mode[s] = a[s]/(a[s]+b[s])
        if round(errors[t], 1) in buckets[s]:
            buckets[s][round(errors[t], 1)].append(behaviors[t][s])
        else:
            buckets[s][round(errors[t], 1)] = [behaviors[t][s]]
        a_post = sum(buckets[s][round(errors[t], 1)])
        b_post = len(buckets[s][round(errors[t], 1)])-a_post
        if sum(buckets[s][round(errors[t], 1)]) > max_buckets[s]:
            max_buckets[s] += 1
            mode_post = a_post/(a_post+b_post)
           # print(behaviors[t])
            print(errors[t])
            print(mode_post)
            print(m_hat[s])
            m_hat[s] = (mode_post - mode[s]) / errors[t]
            #print(m_hat[s])
            #print(M[s])



T: 0, True Priors: [0.7052405  0.43962337], Posterior Estimate: [0.5, 0.5]
True M: [-0.4341728   0.43380865], M Estimate: [0, 0]
0.001926047619715465
1.0
0
0.10400810469286295
1.0
0
-0.014436884995445303
1.0
207.67918503442402
0.13642666399044112
1.0
4.807317674680308
0.09229829663636129
1.0
3.257753517124665
0.007720160059453341
1.0
-25.97513245539525


In [122]:
# just curious to see what happens if we use memory
a_m = [2]*state_size
b_m = [2]*state_size
mode_m = [0.5]*state_size
mem = 15

for t in range(time):
    if t % 10 == 0:
        print(f"Time: {t}, True Priors: {base_priors}, Estimates: {mode_m}")
    for s in range(state_size):
        if t > mem:
            a_m[s] = sum([behaviors[i][s] for i in range(t, t-mem, -1)])
            b_m[s] = mem-a[s]
            mode_m[s] = a_m[s]/(a_m[s]+b_m[s])
        else:
            a_m[s] = a_m[s]+behaviors[t][s]
            b_m[s] = b_m[s]-behaviors[t][s]+1
            mode_m[s] = a_m[s]/(a_m[s]+b_m[s])

Time: 0, True Priors: [0.44036742 0.29804642], Estimates: [0.5, 0.5]
Time: 10, True Priors: [0.44036742 0.29804642], Estimates: [0.5714285714285714, 0.14285714285714285]
Time: 20, True Priors: [0.44036742 0.29804642], Estimates: [-0.1125, 0.0]
Time: 30, True Priors: [0.44036742 0.29804642], Estimates: [-0.08536585365853659, 0.0]
Time: 40, True Priors: [0.44036742 0.29804642], Estimates: [-0.09876543209876543, 0.0]
Time: 50, True Priors: [0.44036742 0.29804642], Estimates: [-0.1125, 0.0]
Time: 60, True Priors: [0.44036742 0.29804642], Estimates: [-0.09876543209876543, 0.0]
Time: 70, True Priors: [0.44036742 0.29804642], Estimates: [-0.08536585365853659, 0.0]
Time: 80, True Priors: [0.44036742 0.29804642], Estimates: [-0.05952380952380952, 0.0]
Time: 90, True Priors: [0.44036742 0.29804642], Estimates: [-0.05952380952380952, 0.0]
Time: 100, True Priors: [0.44036742 0.29804642], Estimates: [-0.09876543209876543, 0.0]
Time: 110, True Priors: [0.44036742 0.29804642], Estimates: [-0.07228915

In [111]:
# let's try to estimate M given error and the estimated prior
